# Generate datasets

The purpose of this notebook is to create two foundational datasets:<br/>
1) admission_diagnosis_table: This dataset contains the most important patient demographic and diagnosis data<br/>
for all admissions.<br/>
2) last_reading: This dataset contains the final reading for 35 chart and lab items for each admission.<br/><br/>
The intention for creating these datasets is that they provide all the data required for selecting the patient<br/>
groups and building the model without the need to directly query the raw data again.<br/><br/>
Both datasets are then saved on AWS S3 to be accessed in subsequent notebooks.

In [1]:
import os
import sys
import pandas as pd

In [2]:
#### MUST DELETE
#import warnings
#warnings.filterwarnings('ignore')
#pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
# Set up paths & import functions
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
src_folder = os.path.join(project_root, 'src')
sys.path.insert(0, src_folder)
from generate_datasets import *
from stats_and_visualisations import *
from s3_storage import *

## admission_diagnosis_table
This dataset includes the most important patient demographic and diagnosis data for all admissions.<br/><br/>
It draws from 3 primary tables in the raw data:<br/>
1) patients - for data specific to patients such as their gender and year of birth.<br/>
2) admissions - to find all admissions associated with each patient.<br/>
3) diagnoses_icd - to find all diagnoses made during each admission.<br/><br/>
The final dataset is at the diagnosis level, meaning there is 1 row per diagnosis per admission.<br/><br/>
Several additional columns are added to the dataset to enable easier analysis:<br/>
1) total_admissions: gives the total number of admissions associated with each subject_id.<br/>
2) admission_number: gives a chronological representation of each admission for each patient, eg 1 would be a<br/>
patient's 1st admission, 2 would be a patient's 2nd admission, etc.<br/>
3) age_on_admission: the difference between dob and admittime. This is correct for all patients under 89, however<br/>
the dob was shifted in the raw data for all patients over 89 to obscure their age to comply with HIIPA.<br/>
Therefore, all patients over 89 are listed as being 89, and an additional flag age_on_admission_shifted was<br/>
created to show when this is the case.<br/>
4) age_adm_bucket: the above age data bucketed into the ranges <45, 45-60, 60-75, 75-89, 89.<br/>
5) ethnicity_simple: cleaned version of the standard ethnicity data to reduce and simplify the ethnicity buckets.<br/> For detailed analysis of ethnicity this shouldn't be used.<br/>

In [4]:
# Create the admission_diagnosis_table and save to S3
admission_diagnosis_table = create_admission_diagnosis_table()
to_s3(obj=admission_diagnosis_table, bucket='mimic-jamesi', filepath='data/admission_diagnosis_table.csv')
admission_diagnosis_table.head(100)

,subject_id,gender,dob,dod,expire_flag,total_admissions,admission_number,hadm_id,entry_diagnosis,age_on_admission,...,age_on_admission_shifted,admittime,dischtime,deathtime,admission_type,ethnicity,ethnicity_simple,hospital_expire_flag,diagnosis_icd9,diagnosis_name
0,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,49322,Ch obst asth w (ac) exac
1,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,51882,Other pulmonary insuff
2,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,41071,"Subendo infarct, initial"
3,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,42841,Ac syst/diastol hrt fail
4,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,99812,Hematoma complic proc
5,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,5849,Acute kidney failure NOS
6,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,42731,Atrial fibrillation
7,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,4660,Acute bronchitis
8,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,4589,Hypotension NOS
9,249,F,2075-03-13,NaT,0,3,1,116935,UNSTABLE ANGINA;ASTHMA;BRONCHITIS,74,...,0,2149-12-17 20:41:00,2149-12-31 14:55:00,NaT,EMERGENCY,WHITE,WHITE,0,41401,Crnry athrscl natve vssl


# first_reading

In [ ]:
# Import chart IDs
item_lookup = from_s3(bucket='mimic-jamesi', filename='item_lookup.csv', index_col=0)
item_lookup

In [ ]:
# Take all readings for these IDs from the raw data
ids = tuple(item_lookup.itemid.tolist())

# Lab events
lab = get_data(query = "SELECT DISTINCT\
                            subject_id\
                            ,hadm_id\
                            ,charttime\
                            ,itemid\
                            ,valuenum\
                        FROM\
                            mimiciii.labevents\
                        WHERE valuenum IS NOT null\
                        AND itemid IN {}".format(ids))

# Lab events
chart = get_data(query = "SELECT DISTINCT\
                            subject_id\
                            ,hadm_id\
                            ,charttime\
                            ,itemid\
                            ,valuenum\
                        FROM\
                            mimiciii.chartevents\
                        WHERE valuenum IS NOT null\
                        AND itemid IN {}".format(ids))

df = lab.append(chart)

# Merge on lookup so that identical concepts can be combined, identified through new_id
df = pd.merge(df, item_lookup[['itemid', 'new_id', 'name']], how='left', left_on='itemid', right_on='itemid')
df.head()

In [ ]:
def compare_itemids(df):

    '''
    This function takes a dataframe containing chart & lab events and outputs visualisations and stats for
    all itemids that are contained. The purpose of this is that if there are multiple itemids that seem to contain
    a similar concept, their values can be compared to see whether this is the case.
    
    The input dataframe must contain the following columns:
    1) itemid: used to identify the chart/ lab event items
    2) valuenum: contains the numerical values of the observations for each itemid
    3) hadm_id: used to identify each admission
    
    The dataframe can be at either the  admission or chart observation level, but the output will reflect this.
    i.e, if the input is at the admission level then the output stats will be at the admission level, whereas if
    the input is at the chart observation level then the output stats will be for every observation recorded across
    all admissions
    
    '''
    
    df.drop_duplicates(inplace=True)    
    df.dropna(inplace=True)

    # Find all itemids so that they can be compared against each other
    item_ids = df.itemid.unique().tolist()

    # --- Plot a KDE: 1 line for each itemid
    plt.figure(figsize = (7, 5))
    for i in item_ids:
        sns.kdeplot(df.loc[df['itemid'] == i, 'valuenum'], label = i)
    plt.ylabel('Density'); plt.title(str(df.name.values[0]));
    plt.show()

    # -- Output stats: Mean, median and standard deviation of the values
    stats = (df.groupby('itemid')
               .agg({'hadm_id': 'nunique',
                     'valuenum': ['mean', 'median', 'std']})
               .reset_index())
    stats.columns = ['itemid', 'patients', 'mean', 'median', 'std']
    print(stats)

    return stats

In [ ]:
def remove_outliers(df, ids, sigma):
    
    '''
    
    This function takes a dataframe of chart observations and removes outliers. It visualises the statistical
    distributions before and after to show the effect of removing the outliers.
    
    The arguments required for the function are:
    1) df: the df containing the chart & lab data
    2) ids: which column in the df contains the identifier that should be used. It should be different to itemid,
       which enables comparison between all itemids assiciated with the new id
    3) sigma: how many standard deviations should be used when identifying outliers.
    
    The input dataframe must be at the chart observation level, and have the following columns:
    1) subject_id
    2) hadm_id
    3) charttime
    4) itemid
    5) valuenum
    6) new_id - the new id that can link multiple itemids (passed in as 'ids')
    7) name - the description of the new_id
    
    '''
    
    new_ids = df[ids].unique().tolist()
    print("Total new IDs: ", len(new_ids))

    # Output df
    cols = ['subject_id', 'hadm_id', 'charttime', 'itemid', 'valuenum', 'new_id', 'name']
    new_df = pd.DataFrame(columns=cols)

    for i in new_ids:
        
        # Find just a single new id and its associated stats. Merge the stats onto the df
        temp_df = df[df['new_id']==i]
        
        print()
        print("=========")
        print(str(temp_df.name.values[0]))
        print("=========")
        print()
        print('Before removing outliers:')
        
        stats = compare_itemids(temp_df)
        stats = stats[['itemid', 'mean', 'std']]
        temp_df = pd.merge(temp_df, stats, how='left', left_on='itemid', right_on='itemid')

        # Find outliers (based on sigma * std dev), setting these to np.nan and then looking at the new distributions
        temp_df['lower'] = temp_df['mean'] - (sigma * temp_df['std'])
        temp_df['upper'] = temp_df['mean'] + (sigma * temp_df['std'])
        temp_df['valuenum'] = np.where((temp_df['valuenum'] > temp_df['lower'])
                                       & (temp_df['valuenum'] < temp_df['upper']),
                                      temp_df['valuenum'], np.nan)
        
        # Re-compare the item_ids to see if removing outliers has improved the distribution match
        # Then remove the outliers and add the remaining observations to the output df
        print()
        print('After removing outliers:')
        compare_itemids(temp_df)
        temp_df.dropna(inplace=True)
        new_df = new_df.append(temp_df[cols])

    # Reset index
    new_df.reset_index(drop=True, inplace=True)

    # QA
    print()
    print("QA STATS:")
    print("Original DF length: ", len(df))
    print("Original unique admissions: ", df.hadm_id.nunique())
    print("New DF length: ", len(new_df))
    print("New unique admissions: ", new_df.hadm_id.nunique())
    
    return new_df

In [ ]:
df = remove_outliers(df=df, ids='new_id', sigma=3)

In [ ]:
# ---- MANUAL CLEANING

# Find equivalent min and max values from Carevue

# Heart_rate
hr_low = df[df['itemid']==211].valuenum.min()
hr_high = df[df['itemid']==211].valuenum.max()

# Respiratory_rate
rr_low = df[df['itemid']==618].valuenum.min()
rr_high = df[df['itemid']==618].valuenum.max()

# Phosphorus
p_low = df[(df['itemid']==827) | (df['itemid']==1534)].valuenum.min()
p_high = df[(df['itemid']==827) | (df['itemid']==1534)].valuenum.max()

# Oxygen saturation
os_low = df[df['itemid']==834].valuenum.min()
os_high = df[df['itemid']==834].valuenum.max()

In [ ]:
# HR (Metavision)
def manual_range_change(df, new_id, low, high):
    df['valuenum'] = np.where((df['valuenum']>high) & (df['new_id']==new_id), np.nan, df['valuenum'])
    df['valuenum'] = np.where((df['valuenum']<low) & (df['new_id']==new_id), np.nan, df['valuenum'])
    compare_itemids(df[df['new_id']==new_id])
    df.dropna(inplace=True)
    return df

In [ ]:
df = manual_range_change(df, 9999018, hr_low, hr_high)
df = manual_range_change(df, 9999025, rr_low, rr_high)
df = manual_range_change(df, 9999022, p_low, p_high)
df = manual_range_change(df, 9999040, os_low, os_high)

In [ ]:
# Get first reading per admission
first_reading = (df.sort_values(by=['subject_id', 'hadm_id', 'new_id', 'charttime'])
                   .groupby(['subject_id', 'hadm_id', 'new_id'])
                   .first()
                   .reset_index())

# Pivot
first_reading=(pd.pivot_table(first_reading, values='valuenum', index=['subject_id', 'hadm_id'], columns='name')
                 .reset_index())
print("Total df length: ", len(first_reading))

# Cleaning - ensure hadm_id is an int
first_reading['hadm_id'] = first_reading['hadm_id'].astype(int)

to_s3(obj=first_reading, bucket='mimic-jamesi', filename='first_reading.csv')
first_reading.head(100)